In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/expanse/lustre/projects/csd819/a3murali/cache/'

from huggingface_hub import login
login(token = "<hf_token>")

import transformers
import torch

import csv
import numpy as np
import pandas as pd

print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForCausalLM, pipeline, LlamaForCausalLM

model = "ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M"
tokenizer = AutoTokenizer.from_pretrained(model)


llamatales_pipeline = pipeline("text-generation", model = model, device = "cuda")

# prompts = {
#     1: "Once upon a time there was a dragon", 
#     2: "Once upon a time there was a princess", 
#     3: "Once upon a time there were two children",
#     4: "Once upon a time there was a prince",
#     5: "Once upon a time there was a frog",
#     6: "Once upon a time there was a king",
#     7: "Once upon a time there was a queen",
#     8: "Once upon a time there was a wolf",
#     9: "Once upon a time there was a genie",
#     10: "Once upon a time there was a poor boy"
# }
prompts = {1: "Once upon a time there was a dragon"}

data = []
for prompt_id in prompts:
    #generation - generate stories for each prompt
    for i in range(2):
        sequences = llamatales_pipeline(
            prompts[prompt_id],
            do_sample = True,
            top_k = 10,
            num_return_sequences = 1,
            max_new_tokens = 512
        )
        generated_story = sequences[0]['generated_text']
        print(generated_story)

        num_tokens_generated_story = len(tokenizer.encode(generated_story))

        #test - collect hidden states
        
        config = AutoConfig.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325", output_hidden_states = True)
        model = AutoModelForCausalLM.from_config(config).to('cuda')
        if(i == 0):
            print(model)
            
            for j in model.named_parameters():
                print(f"{j[0]} -> {j[1].device}")


        tokenizer = AutoTokenizer.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325/", config = config)

        inputs = tokenizer(generated_story, return_tensors="pt").to("cuda")

        outputs = model.generate(inputs.input_ids, attention_mask=inputs["attention_mask"], do_sample = True, top_k = 10, num_return_sequences = 1, max_new_tokens = 512, eos_token_id = tokenizer.eos_token_id, pad_token_id = tokenizer.pad_token_id, return_dict_in_generate=True)
        
        #analysis - collect prompt id of each token
        output_id = []
        for token in range(num_tokens_generated_story):
            output_id.append(prompt_id)
        
        convert_hidden_states = []
        print(type(outputs))

        for i in outputs.hidden_states:
            convert_hidden_states.append([j.detach().cpu().numpy() for j in i])
        
        print(len(convert_hidden_states))
        print(len(convert_hidden_states[0]))
        print(len(convert_hidden_states[0][0]))
        print(len(convert_hidden_states[0][0][0]))
        print(len(convert_hidden_states[0][0][0][0]))
        #print(len(convert_hidden_states[0][0][0][0][0]))
        data.append([prompts[prompt_id], generated_story, convert_hidden_states, np.array(output_id)])

        #(([]))

df = pd.DataFrame(data, columns = ["prompt", "story", "hidden_states", "output_token_prompt_id"], dtype = 'object')
df

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/a3murali/.cache/huggingface/token
Login successful
PyTorch Version: 1.13.0a0+d321be6
Transformers Version: 4.35.0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Once upon a time there was a dragon named Timmy. Timmy lived in a cave with his best friend, a little rabbit named Rosie. Timmy was a very happy dragon. He loved to play with his best friend.

One day, Timmy's cave started to smell very bad. He could not see very well or breathe fire. Timmy was worried that he had been in trouble. He asked Rosie, "What's wrong?" Rosie looked at him and said, "The dragon of smoke is coming out of your cave." Timmy didn't know what Rosie meant by "naughty" or "nosy." But he felt a little scared.

Timmy and Rosie decided to go outside and see what was happening. They looked through the cave entrance and saw smoke everywhere. They saw that all the trees were on fire. Timmy's heart was racing fast. He was worried his cave was going to get in trouble. Rosie said, "Don't worry, Timmy. We'll get out of here." They walked carefully outside, away from the fire.

As they walked out, they saw the villagers working together to clean up the fire. Timmy was relieved 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (512). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<class 'transformers.generation.utils.SampleDecoderOnlyOutput'>
512
9
1
365
512
Once upon a time there was a dragon named Firedog. Firedog loved to play outside in the sunshine. He would run and jump and roar loudly, "Roar!" to make all the children in the village wake up.

Firedog lived in a big cave with a friendly owner named Lily. Lily took good care of Firedog by giving him yummy food and a fresh drink of water every day. She was a very dependable friend to Firedog. But one day, a big storm came to the village. The strong wind knocked over a big tree, making a big crash in the cave. Lily tried to move the tree but it was too heavy.

The storm got stronger and stronger, and the village got messy. Fierce winds knocked over big things, like the roof and the walls. The villagers cried out, "Help! Help!" But Fog was too weak to help. He got stuck in the falling tree branches. The villagers tried to get him out, but it was too late. Fierce winds had taken Farks from the village and they

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<class 'transformers.generation.utils.SampleDecoderOnlyOutput'>
512
9
1
229
512


,prompt,story,hidden_states,output_token_prompt_id
0,Once upon a time there was a dragon,Once upon a time there was a dragon named Timm...,[[[[[ 0.01972523 0.02009857 0.02478487 ... ...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Once upon a time there was a dragon,Once upon a time there was a dragon named Fire...,[[[[[ 0.02893161 -0.02468858 -0.01625257 ... ...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [2]:
# df.to_csv("story_dataset.csv", index = False)